In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.day_to_day import S_driver_opt_out_TS, S_traveller_opt_out_TS
from MaaSSim.day_to_day import d2d_kpi_veh, d2d_kpi_pax

In [4]:
import osmnx as ox
import networkx as nx
import random
import math
import pandas as pd
import matplotlib.pyplot as plt          
import numpy as np
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

In [5]:
params = MaaSSim.utils.get_config('glance.json')
params.city = "Amsterdam, Netherlands"
params.paths.G = "../../data/graphs/Amsterdam.graphml"
params.paths.skim = "../../data/graphs/Amsterdam.csv"
params.vehicle_fixed_positions = True
params.times.patience = 600 #second
params.dist_threshold = 20000 #meter
params.dist_threshold_min = 3000 #meter

params.d2d.probabilistic = True
params.d2d.heterogeneous = False
params.d2d.B_income = 1
params.d2d.fuel_cost = 0.25 #euro/km

params.d2d.B_inveh_time = 1
params.d2d.B_exp_time = 2 #1.5 in oded's paper
params.PT_fare = 1 ## euro
params.PT_avg_speed = 6

params.nPM = 2 # number of platforms. platform id starts from 1
params.platforms.base_fare = 1.4 #euro
params.platforms.fare = float(1.20) #euro/km
params.platforms.min_fare = float(2) # euro
params.platforms.comm_rate = float(0.20)
params.platforms.discount = float(0.0)
params.platforms.daily_marketing = True

params.d2d.learning_d = 1 
params.d2d.B_Experience = 0.80
params.d2d.B_WOM = 0.20
params.d2d.B_Marketing = 0.0
params.d2d.diffusion_speed = 10/100 # speed for M is twice greater than WOM.
params.d2d.m = 5
params.d2d.mn = 100/12 # for NL
params.d2d.ini_att = 0.01
params.d2d.Eini_att = 0.01
params.d2d.adj_s = 2
params.VoT = 10.63 # value of time per hour
params.assert_me = False
params.seed = 1
params.paths.requests = 'Amsterdam_requests.csv'
params.simTime = 4
params.d2d.res_wage = params.simTime*10.63 #euro

In [11]:
params.threshold_u = 0.005
params.max_revenue = 4000 # maximum revenue with the initial fare
params.alpha = 0.8
params.initial_capital = 50000
params.expense_per_day = 500
params.random_ini_position = False
params.step_size = 0.2 # euro/km
params.min_fare, params.max_fare  = 0.0, 3.0
params.initial_fares = (1.0,1.0)

params.nD = 2#1000
params.turnover_interval = 2#50 #---------------------------------
params.nP = 2000
params.nV = 200

In [12]:
sim = MaaSSim.simulators.Try_and_Select(params=params, f_driver_out=S_driver_opt_out_TS,f_trav_out=S_traveller_opt_out_TS,
                                  kpi_veh=d2d_kpi_veh,kpi_pax=d2d_kpi_pax, logger_level=logging.WARNING)

This simulation uses albatros data
29-12-24 11:30:43-WARNING-Setting up 4h simulation at 2024-12-29 09:29:58 for 200 vehicles and 2000 passengers in Amsterdam, Netherlands
============ P1 TURN ============
29-12-24 11:30:58-WARNING-simulation time 4.9 s
Day =  0 --- on the Left cell with fare 0.8
(np1,vp1) =  (0, 0)     (np2,vp2) =  (0, 0)
--------------------------------------------
29-12-24 11:31:15-WARNING-simulation time 6.4 s
Day =  1 --- on the Left cell with fare 0.8
(np1,vp1) =  (9, 2)     (np2,vp2) =  (20, 1)
--------------------------------------------
--------------------------------------------
P1:  (0.8, 0.00321272, 0.0022034000000000003, 0.0072499999999999995)
29-12-24 11:31:31-WARNING-simulation time 4.1 s
Day =  0 --- on the Middle cell with fare 1.0
(np1,vp1) =  (0, 0)     (np2,vp2) =  (0, 0)
--------------------------------------------
29-12-24 11:31:45-WARNING-simulation time 5.3 s
Day =  1 --- on the Middle cell with fare 1.0
(np1,vp1) =  (17, 3)     (np2,vp2) =  (1

In [14]:
sim.res[1].plat_exp

,fare,revenue,profit,nP,nV,min_wage_sub,remaining_capital,P_market_share,V_market_share,market_share
platform_id,,,,,,,,,,
1,1.0,24.075,-552.064,17,3,76.1390,48947.936,0.0085,0.015,0.01175
2,1.0,16.1418,-613.4887,12,4,129.6305,48886.5113,0.006,0.02,0.013


In [51]:
df = sim.res[1].veh_exp

sum(df[df.platform_id==2].REVENUE-df[df.platform_id==2].COST)

40.4495

In [52]:
(sum(df[df.platform_id==2].REVENUE)*(100/80)*0.2) -

16.128

In [57]:
veh = sim.res[1].veh_exp
veh[veh.ACTUAL_INC>0].ACTUAL_INC

veh
60     42.52
126    42.52
128    42.52
138    42.52
153    42.52
174    42.52
180    42.52
Name: ACTUAL_INC, dtype: float64

In [58]:
params.d2d.res_wage

42.52

In [23]:
sum(df[df.platform_id==1].MIN_WAGE_SUB)

76.13900000000001

56.887800000000006

pax
12      2.4700
46      1.2000
87      1.3324
175     1.1120
237     0.0000
578     2.2264
598     1.0952
623     2.3686
642     2.0064
683     3.0794
945     1.1612
996     1.2166
1065    1.0910
1111    0.8880
1128    0.8936
1614    0.9610
1636    0.9732
Name: plat_revenue, dtype: float64

,veh,pos,t,event,paxes
0,138,46391478.0,0,STARTS_DAY,[]
1,138,46391478.0,0,DECIDES_NOT_TO_DRIVE,[]
